In [1]:
import torch,os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

True
cuda:0


In [2]:
model_name = "Llama-2"
model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"

In [3]:
#max_length_input=196
lr = 2e-4
num_epochs = 5
batch_size = 4

# Model init

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline, AutoModelForSeq2SeqLM,LlamaTokenizerFast
from langchain.llms import HuggingFacePipeline
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,cache_dir="Thin/llms")
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,torch_dtype=torch.bfloat16,
                                              device_map="auto",cache_dir="Thin/llms")

/home/itp1hc/.conda/envs/thinSA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-28 07:39:16.988899: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 07:39:16.989003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 07:39:17.020641: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 07:39:17.109042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow bi

# Run experiments

# Dataset

In [5]:
path_train = 'dataset/Homomexican/track_3_train.csv'
path_dev = 'dataset/Homomexican/track_3_dev.csv'
path_test = 'dataset/Homomexican/track_3_public_test.csv'
path_output = "./Output/SA/HSA/"

import pandas as pd
data_train = pd.read_csv(path_train)
data_train.reset_index(drop=True, inplace=True)
#id_train = data_train['id_EXIST'].tolist()

data_test= pd.read_csv(path_test)
data_test.reset_index(drop=True, inplace=True)
id_test = data_test["sub_id"].tolist()

data_dev= pd.read_csv(path_dev)
data_dev.reset_index(drop=True, inplace=True)
#id_dev = data_dev["id_EXIST"].tolist()

data_train.head()

,lyrics,label
0,[Verso 1]\nNo sé si tu boca está besando a otr...,NP
1,El tiempo se ha pasado\nY yo no sé nada de ti\...,NP
2,Me haces sentir que quiera ser mejor.\nMe hace...,NP
3,"[Introducción]\nWu! Ja, ja, wu\n[Verso]\nYa no...",NP
4,Yo me pregunto porque sera\nNos gusta sufrir\n...,NP


# Preprocessing tweet

In [6]:
def split_part(my_string):    
    #my_string = "This is a string\nwith multiple lines\nand we want to split it."

    # Split the string by newlines
    lines = my_string.splitlines()

    # the desired number of parts
    num_parts = 4

    # Split the lines into parts of approximately equal length
    parts = []
    for i in range(num_parts-1):
        start = i * (len(lines) // num_parts)
        end = (i + 1) * (len(lines) // num_parts)
        parts.append('\n'.join(lines[start:end]))

    start = 3 * (len(lines) // num_parts)
    end = len(lines)
    parts.append('\n'.join(lines[start:end]))
    return parts

In [7]:
data = data_train[data_train["label"]=='P']
new_lyrics = []
label = ["P"]*234
for index, item in enumerate(data["lyrics"]):
    part = split_part(item)
    for i in part:
        new_lyrics.append(i)

In [8]:
label = ["P"]*len(new_lyrics)

In [9]:
augu_data = pd.DataFrame({'lyrics': new_lyrics,
                      'label': label})
augu_data.head()

,lyrics,label
0,[Intro: Babo]\nVolvió el Don Vergas\nBien rela...,P
1,"Eso lo saben de sobra, bien sabido desde siemp...",P
2,No me diga dónde hay si yo soy el que la trae\...,P
3,"Vaya-vaya (Vaya-vaya), oye-oye (Oye-oye)\nUn d...",P
4,Muy muy lejos de aquí\nEn una galaxia\nEl pode...,P


In [10]:
data_train = pd.concat([data_train, augu_data])
len(data_train)

1140

In [11]:
data_train["label"].tolist().count("P")

195

In [12]:
#Instruction + output prompt contain all task
import re
def create_instruction_input_output(df):
    input_text = []
    output_text = []
    inputs = df["lyrics"].tolist()
    output = df["label"].tolist()
    for index, item in enumerate(inputs):     
        item = re.sub(r"[\r\n]+", ". ", item) #replace all type of white space by a single space   
        
        prompt_new = f"""[INST]
Classify the sentiment of a following lyrics from a song:
{item[:1000]}
## if the lyrics directed against any person whose sexual orientation and/or gender identity differs form cis-heterosexuality, output "P"
## if the lyrics not include any hate speech against the LGBT+ population but do mention this comunity, output "NP".
Answer: [/INST]"""     
        input_text.append(prompt_new)
        output_text.append(output[index])
    print(len(input_text),len(output_text))
    return input_text,output_text

In [13]:
# Create instruction input ouput for each task
input_train,output_train = create_instruction_input_output(data_train)
print(input_train[3])
print(output_train[3])

1140 1140
[INST]
Classify the sentiment of a following lyrics from a song:
[Introducción]. Wu! Ja, ja, wu. [Verso]. Ya no te haré llorar, ya no te haré sufrir. Prometo seriamente no volver a molestarte. Ni andar tocando tu ventana para hablarte. Duerme tranquila ya no vuelvo a molestarte. [Puente]. Tú se feliz con quien tú quieras, mientras tanto. Yo voy tratando poco a poco de olvidarte. [Pre-Estribillo]. Fueron tan hermosas. Todas esas noches, todos esos días. Que sinceramente. Fue lo más hermoso de toda mi vida. [Estribillo]. Qué lástima que nuestro amor haya acabado. Y que feliz me sentiré cuando te vea. Que eres feliz con otro amor y que te quiera. Pues tú mereces ser feliz con quien tú quieras. [Interludio]. ¡Y arriba Juárez!. ¡Arriba!. [Pre-Estribillo]. Fueron tan hermosas. Todas esas noches, todos esos días. Que sinceramente. Fue lo más hermoso de toda mi vida. [Estribillo]. Qué lástima que nuestro amor haya acabado. Y que feliz me sentiré cuando te vea. Que eres feliz con otro

In [14]:
train_df = pd.DataFrame(list(zip(input_train, output_train)), columns =['text', 'label'])
# # removing rows that contain "UNKNOWN" OR ""-""
# train_df = train_df[(train_df["label"] != "-") & (train_df["label"] != "UNKNOWN")]
train_df.head(6)

,text,label
0,[INST]\nClassify the sentiment of a following ...,NP
1,[INST]\nClassify the sentiment of a following ...,NP
2,[INST]\nClassify the sentiment of a following ...,NP
3,[INST]\nClassify the sentiment of a following ...,NP
4,[INST]\nClassify the sentiment of a following ...,NP
5,[INST]\nClassify the sentiment of a following ...,NP


In [15]:
# create a dictionary datatype contain train, val set
from datasets import Dataset, DatasetDict

d1s = Dataset.from_pandas(train_df)

dataset = DatasetDict()
dataset['train'] = d1s
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1140
    })
})


# MODEL

In [16]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [17]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [18]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8, # Lora attention dimension.
        lora_alpha=16, # the alpha parameter for Lora scaling.
        lora_dropout=0.05, # the dropout probability for Lora layers.
        bias="none",
        target_modules=[
            #"embed_tokens",
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
            #"lm_head",
        ]
)

model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 19988480 || all params: 6758404096 || trainable%: 0.2957573965106688


In [19]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

# data preprocessing
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
# max_length = max([len(tokenizer(review)["input_ids"]) for review in df1["x_input"][100:200].tolist()])
# print(max_length)
max_length = 600

In [20]:
def preprocess_function(examples):
    batch_size = len(examples["text"])
    inputs = [item + " " for item in examples["text"]]
    targets = examples["label"]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]

        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (max_length - len(sample_input_ids)) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs["attention_mask"][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset[f"train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

#train_dataset = processed_datasets["train"]
train_dataset = processed_datasets[f"train"]
#eval_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
#eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)


Running tokenizer on dataset: 100%|██████████| 1140/1140 [00:00<00:00, 2828.61 examples/s]


In [21]:
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

# training and evaluation
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        #         print(batch)
        #         print(batch["input_ids"].shape)
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # model.eval()
    # eval_loss = 0
    # eval_preds = []
#     for step, batch in enumerate(tqdm(eval_dataloader)):
#         batch = {k: v.to(device) for k, v in batch.items()}
#         with torch.no_grad():
#             outputs = model(**batch)
#         loss = outputs.loss
#         eval_loss += loss.detach().float()
#         eval_preds.extend(
#             tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
#         )

#     eval_epoch_loss = eval_loss / len(eval_dataloader)
#     eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=}") #{eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 285/285 [02:54<00:00,  1.64it/s]


epoch=0: train_ppl=tensor(1.2057, device='cuda:0') train_epoch_loss=tensor(0.1870, device='cuda:0')


100%|██████████| 285/285 [02:52<00:00,  1.65it/s]


epoch=1: train_ppl=tensor(1.0670, device='cuda:0') train_epoch_loss=tensor(0.0648, device='cuda:0')


100%|██████████| 285/285 [02:52<00:00,  1.65it/s]


epoch=2: train_ppl=tensor(1.0274, device='cuda:0') train_epoch_loss=tensor(0.0270, device='cuda:0')


100%|██████████| 285/285 [02:52<00:00,  1.65it/s]


epoch=3: train_ppl=tensor(1.0065, device='cuda:0') train_epoch_loss=tensor(0.0065, device='cuda:0')


100%|██████████| 285/285 [02:52<00:00,  1.65it/s]

epoch=4: train_ppl=tensor(1.0022, device='cuda:0') train_epoch_loss=tensor(0.0022, device='cuda:0')


# Prediction

In [22]:
def evaluate_model(sample,max_len_input=max_length):
    inputs = tokenizer(sample, return_tensors="pt").to('cuda')
    generation_config  = GenerationConfig(
    do_sample=True,
    max_new_tokens=128,
    top_k=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    penalty_alpha = 0.6,
    return_full_text=False,
    )
    
    outputs = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    output = output[0].split("[/INST]")[1].strip()
    return output

In [31]:
#Instruction + output prompt contain all task
import re
def create_instruction_input(df):
    input_text = []
    inputs = df["lyrics"].tolist()
    for index, item in enumerate(inputs):     
        item = re.sub(r"[\r\n]+", ". ", item) #replace all type of white space by a single space   
        
        prompt_new = f"""[INST]
Classify the sentiment of a following lyrics from a song:
{item[:1000]}
## if the lyrics directed against any person whose sexual orientation and/or gender identity differs form cis-heterosexuality, output "P"
## if the lyrics not include any hate speech against the LGBT+ population but do mention this comunity, output "NP".
Answer: [/INST]"""
        input_text.append(prompt_new)
    print(len(input_text))
    return input_text

## PREDICTION ON TEST

In [32]:
test_input = create_instruction_input(data_test)

246


In [33]:
from tqdm import tqdm

y_pred = []

#predict one batch of datadev at a time
for sample in tqdm(test_input):
    pred = evaluate_model(sample)
    y_pred.append(pred)

print(len(y_pred))
print(y_pred[:5])

100%|██████████| 246/246 [00:52<00:00,  4.66it/s]

246
['NP', 'NP', 'NP', 'NP', 'NP']


In [34]:
df_test = pd.DataFrame({'sub_id': id_test,
                      'label': y_pred})
df_test.head()

,sub_id,label
0,0_Track3,NP
1,1_Track3,NP
2,2_Track3,NP
3,3_Track3,NP
4,4_Track3,NP


In [35]:
df_test.to_csv(f'homomex31test.csv')

In [36]:
print(y_pred.count("P"), y_pred.count("NP"))
print(y_pred.count("P") + y_pred.count("NP"))

2 244
246


In [37]:
len(y_pred)

246

## PREDICTION ON DEV

In [25]:
dev_input = create_instruction_input(data_dev)
dev_output = data_dev["label"].tolist()

600


In [26]:
y_pred_dev = []

#predict one batch of datadev at a time
for sample in tqdm(dev_input):
    pred = evaluate_model(sample)
    y_pred_dev.append(pred)

print(len(y_pred_dev))
print(y_pred_dev[:5])

100%|██████████| 600/600 [02:05<00:00,  4.80it/s]

600
['NP', 'NP', 'NP', 'NP', 'NP']


In [27]:
with open(f'homomex31dev.txt', 'w') as f: 
    for word in y_pred_dev: 
        f.write(f'{word}\n') 

In [28]:
len(y_pred_dev)

600

In [29]:
print(y_pred_dev.count("P"), y_pred_dev.count("NP"))
print(y_pred_dev.count("P") + y_pred_dev.count("NP"))

32 568
600


-----------------------------------------------------------------------------------------------------------------

In [30]:
from sklearn.metrics import *
print(classification_report(data_dev["label"].tolist(), y_pred_dev))

              precision    recall  f1-score   support

          NP       0.95      0.96      0.96       560
           P       0.38      0.30      0.33        40

    accuracy                           0.92       600
   macro avg       0.66      0.63      0.65       600
weighted avg       0.91      0.92      0.92       600

